In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [3]:
!pip install datasets


  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [4]:
from datasets import load_dataset

dataset = load_dataset("conll2003")
train_data = dataset['train']
val_data = dataset['validation']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [7]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en")

db = DocBin()
for example in train_data:
    text = example['tokens']
    annotations = example['ner_tags']

    entities = []
    start_char = 0
    for i, (token, ner_tag) in enumerate(zip(text, annotations)):
        end_char = start_char + len(token)
        if ner_tag != 0:
            entities.append((start_char, end_char, nlp.vocab.strings[ner_tag]))
        start_char = end_char + 1

    doc = nlp(" ".join(text))
    ents = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./train.spacy")
db.to_disk("val.spacy")

In [10]:
from google.colab import files
files.download("train.spacy")
files.download("val.spacy")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
!python -m spacy train /content/config.cfg --paths.train /content/train.spacy --paths.dev /content/val.spacy --output /content/output


ℹ Saving to output directory: /content/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.26    0.00    0.00    0.00    0.00
  0     200         87.63   3469.77   58.54   60.03   57.12    0.59
  0     400        322.63   2210.70   77.25   77.50   77.00    0.77
  0     600        306.83   2083.39   83.58   84.33   82.84    0.84
  0     800        312.63   2115.00   87.83   88.36   87.30    0.88
  0    1000        357.05   2230.82   90.88   91.05   90.71    0.91
  1    1200        517.91   2187.74   93.64   94.10   93.18    0.94
  1    1400        410.69   1650.70   95.06   94.83   95.30    0.95
  1    1600        587

In [24]:
!python -m spacy evaluate /content/output/model-last /content/val.spacy


ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   99.87 
NER R   99.91 
NER F   99.89 
SPEED   20645 


=============================== NER (per type) ===============================

                P        R        F
IS_DIGIT    99.78    99.94    99.86
IS_TITLE    99.71    99.91    99.81
IS_ALPHA    99.94    99.94    99.94
IS_ASCII    99.93    99.98    99.96
IS_PUNCT    99.93    99.89    99.91
IS_LOWER    99.78   100.00    99.89
IS_UPPER    99.91    99.13    99.52
IS_SPACE   100.00   100.00   100.00



In [26]:
nlp = spacy.load("/content/output/model-last")

text = "Apple Inc. is planning to release the iPhone 15 next month in New York"
doc = nlp(text)


for ent in doc.ents:
    print(ent.text, ent.label_)

Apple IS_DIGIT
Inc. IS_LOWER
New IS_PUNCT
York IS_SPACE
